# INF-0618 - Deep Learning - Trabalho 2

---


## Transformers
Professora:  -- wanner@unicamp.br

Instituto de Computação - Unicamp 2023

* Thais Borges Damacena
* Ricardo Augusto Ferrari


# Tradução Automática de Textos Inglês-Francês com Transformer

Nessa prática, vamos tentar criar um tradutor automático de textos inglês-francês. Para isto, usaremos modelo Transformer de forma a lidar com as informações de texto.   

In [9]:
import pickle
import random

with open("ParaCrawl99k/ParaCrawl99K_EnPt_PCrawlGoogleT.pkl", "rb") as f:
        dataset = pickle.load(f)

random.shuffle(dataset)


In [36]:
# Embaralha os dados e divide em 80% para treino, 10% para validação e 10% para teste
nData = len(dataset)

nTrain = int(nData * 0.8)  
nVal = int(nData * 0.1)  
train = dataset[:nTrain]
val = dataset[(nTrain + 1):(nTrain + nVal)]
test = dataset[(nTrain + nVal + 1): (nTrain + 2 * nVal)]


('So he thought That Should he give something to the little boy.',
 'Então ele pensou que ele deveria dar algo para o menino.')

In [41]:
# !pip install sentencepiece

# Tokenizer 
from transformers import T5Tokenizer

# Tensorflow (bare model, baremodel + language modeling head)
from transformers import TFT5Model, TFT5ForConditionalGeneration

model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'

tokenizer = T5Tokenizer.from_pretrained(model_name)


# TensorFlow
model_tf = TFT5ForConditionalGeneration.from_pretrained(model_name)


TypeError: 'NoneType' object is not callable

O próprio objeto `dataset` é um objeto [` DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), que contém uma chave para o conjunto de treinamento, validação e teste (quando houver):

In [ ]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

Neste caso, temos apenas um conjunto de treino e, a partir deste, iremos criar um conjunto de treino e validação:

In [ ]:
books = books["train"].train_test_split(test_size=0.2)

Vamos visualizar a nova estrutura da base de dados

In [ ]:
books

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})

Para acessar um elemento real, você precisa primeiro selecionar uma divisão e, em seguida, fornecer um índice:

In [ ]:
books['train'][14]

{'id': '90177',
 'translation': {'en': 'Often these powerful animals rushed at the lounge window with a violence less than comforting.',
  'fr': 'Souvent, ces puissants animaux se précipitaient contre la vitre du salon avec une violence peu rassurante.'}}

Para ter uma ideia de como os dados se parecem, a função a seguir mostrará alguns exemplos escolhidos aleatoriamente no conjunto de dados.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(books["train"])

,id,translation
0,12144,"{'en': '""Entirely.""', 'fr': '-- Entièrement.'}"
1,100634,"{'en': 'And who knows what will become of the survivor of us after a long solitude on this island?', 'fr': 'Et qui sait ce que deviendrait le dernier vivant de nous, après une longue solitude sur cette île?'}"
2,96927,"{'en': 'As to Pencroft, he had sailed over every sea, a carpenter in the dockyards in Brooklyn, assistant tailor in the vessels of the state, gardener, cultivator, during his holidays, etc., and like all seamen, fit for anything, he knew how to do everything.', 'fr': 'Quant à Pencroff, il avait été marin sur tous les océans, charpentier dans les chantiers de construction de Brooklyn, aide- tailleur sur les bâtiments de l'état, jardinier, cultivateur, pendant ses congés, etc., et comme les gens de mer, propre à tout, il savait tout faire.'}"
3,63532,"{'en': 'Gudule replied in a careless tone,−−', 'fr': 'Gudule répondit d’un ton insouciant :'}"
4,76202,"{'en': 'Elle trouva sur le pont du fossé de la citadelle le général Fontana et Fabrice, qui sortaient à pied.', 'fr': 'On the bridge over the moat of the citadel she met General Fontana and Fabrizio, who were coming out on foot.'}"


# Pré-processando os dados 

Uma abordagem comum em Processamento de Linguagem Natural (NLP) é usar um dicionário/vocabulário para codificar as palavras presentes em um texto. Há diferentes maneiras de construir o dicionário, mas essencialmente gostaríamos de incluir as palavras mais significativas do nosso conjunto de treino (assumindo que ele vai generalizar bem para o conjunto de teste). Cada palavra em uma passagem de texto será codificada (transformada) em um inteiro associado a uma palavra no nosso dicionário.

Por exemplo, suponha que nosso dicionário é:

{"movie":1, "actor": 2, "actress": 3, "cool":4, "bad":5, "action":6 ... "awesome": 100 ...}

Associamos a palavra movie (a palavra mais comum no nosso conjunto de treinamento) ao número 1, bad ao número 5 e assim por diante. Agora, suponha que tenhamos as seguintes duas frases (já desconsideramos as palavras que não estão no nosso dicionário).

    Frase 1: "movie awesome. Cool actor."

    Frase 2: "movie bad. Awesome actor."

Eles serão codificados como:

    Codificação 1: [1,100,4,2]

    Codificação 2: [1,5,100,2]

É importante lembrar que não precisamos construir este dicionário do zero, visto que vamos utilizar o dicionário aprendido pelo modelo T5. Para fazer tudo isso, instanciamos nosso tokenizer com o método `AutoTokenizer.from_pretrained`, que garantirá:

- obtemos um tokenizer que corresponde à arquitetura do modelo que queremos usar,
- baixamos o vocabulário (dicionário) usado durante o pré-treinamento neste ponto de verificação específico.

Esse vocabulário será armazenado em cache, portanto, não será baixado novamente na próxima vez que executarmos o celular.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

Em vez de uma frase, podemos passar uma lista de frases:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

Para preparar os valores alvos para nosso modelo, precisamos tokenizá-los dentro do gerenciador de contexto `as_target_tokenizer`. Isso garantirá que o tokenizer use os tokens especiais correspondentes aos destinos:

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


Podemos então escrever a função que irá pré-processar nossas amostras. Nós apenas os alimentamos com o `tokenizer` com o argumento` truncation = True`. Isso garantirá que uma entrada mais longa do que o modelo selecionado será truncada para o comprimento máximo aceito pelo modelo. O preenchimento será tratado mais tarde (em um Data Collator), portanto, preenchemos os exemplos de entrada até o tamanho indicado (parâmetro `max_length`=128 tokens) para termos um padrão na entrada do modelo. Entradas que geraram mais de 128 tokens serão truncadas em 128.

O modelo T5 requer um prefixo especial para colocar antes das entradas, sendo assim, devemos adaptar a célula a seguir.

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Esta função funciona com um ou vários exemplos. No caso de vários exemplos, o tokenizer retornará uma lista de listas para cada chave:

In [ ]:
preprocess_function(books['train'][:2])

{'input_ids': [[13959, 1566, 12, 2379, 10, 6006, 47, 2764, 323, 12, 8, 182, 3, 51, 6770, 13, 112, 12432, 5, 1], [13959, 1566, 12, 2379, 10, 37, 4228, 18, 17773, 4205, 130, 20, 9, 89, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[6006, 4104, 42, 5398, 1092, 2623, 4007, 15, 247, 50, 2288, 693, 93, 3, 32, 7, 5, 1], [622, 9593, 9, 11348, 20, 3, 17, 21813, 3, 9998, 3, 7, 1211, 1395, 5, 1]]}

Para aplicar esta função em todos os pares de sentenças em nosso conjunto de dados, apenas usamos o método `map` do nosso objeto` conjunto de dados` que criamos anteriormente. Isso aplicará a função em todos os elementos de todas as divisões no `dataset`, então nossos dados de treinamento, validação e teste serão pré-processados em um único comando.

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

  0%|          | 0/102 [00:00<?, ?ba/s]

  0%|          | 0/26 [00:00<?, ?ba/s]

Os resultados são armazenados em cache automaticamente pela biblioteca Datasets para gasto excessitvo de tempo nesta etapa na próxima vez em que executarmos o notebook. A biblioteca Datasets normalmente é inteligente o suficiente para detectar quando a função passada ao `map` mudou (e, portanto, requer não usar os dados de cache). Por exemplo, ele detectará corretamente se você alterar a tarefa na primeira célula e executar o notebook novamente. Datasets avisa quando usa arquivos em cache, você pode passar `load_from_cache_file = False` na chamada para `map` para não usar os arquivos em cache e forçar o pré-processamento a ser aplicado novamente.

Observe que passamos `batched = True` para codificar os textos por lotes juntos. Isso é para aproveitar todos os benefícios do tokenizer rápido que carregamos anteriormente, que usará multithreading para tratar os textos em um lote simultaneamente.

# Fine-tuning do Modelo

Agora que nossos dados estão prontos, podemos baixar o modelo pré-treinado e ajustá-lo. Como nossa tarefa é do tipo sequência a sequência, usamos a classe `AutoModelForSeq2SeqLM`. Como com o tokenizer, o método `from_pretrained` irá baixar e armazenar em cache o modelo para nós.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Para a etapa de ajuste do modelo, em TensorFlow, comece convertende os seus conjuntos de dados para o formato `tf.data.Dataset` com to_tf_dataset. Especifique as entradas e os rótólos em colunas, se deve embabaralhar a ordem do conjunto de dados, o tamanho do batch, e alguma função de processamento dos dados (collate):

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

tf_train_set = tokenized_books["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=64,
    collate_fn=data_collator,
)

tf_val_set = tokenized_books["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=64,
    collate_fn=data_collator,
)

Por fim, precisamos definir qual o otimizador será utilizado, assim como os seus respectivos hyperparâmetros. 

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Tudo pronto! Agora só precisamor compilar o modelo para verificar se está tudo correto e, posteriormente, executarmos o treinamento. 

In [ ]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=3)

Epoch 1/3
1588/1588 [==============================] - 3729s 2s/step - loss: 1.9749 - val_loss: 1.6706
Epoch 2/3
1588/1588 [==============================] - 3697s 2s/step - loss: 1.8564 - val_loss: 1.6138
Epoch 3/3
1588/1588 [==============================] - 3699s 2s/step - loss: 1.8036 - val_loss: 1.5805


Uma vez que o modelo está treinado, é interessante investigar como estão as predições do mesmo. Como utilizamos a Huggingface, uma das formas mais diretas de investigar as predições do modelo é através da criação de um `pipeline`, cujos parâmetros indicam qual tarefa será realizada (translation_en_fr), um modelo e um tokenizador.

In [ ]:
from transformers import pipeline

en_fr_translator = pipeline(f"translation_{source_lang}_to_{target_lang}", 
                            model=model,
                            tokenizer=tokenizer)

In [ ]:
en_fr_translator("How old are you?")

[{'translation_text': ' quel âge êtes-vous?'}]

Agora, iremos verificar as predições do modelo treinado com 5 amostras aleatórios que estão no conjunto de validação.

In [ ]:
import numpy as np

for val_sample in np.random.choice(books['test'], size=5):
  original = val_sample['translation'][source_lang]
  target_text = val_sample['translation'][target_lang]
  translation = en_fr_translator(original)
  print(f"Original Text: {original} \n Prediction: {translation[0]['translation_text']} \n Answer: {target_text}")
  print(30*"--")

Original Text: La duchesse avait moins que jamais oublié sa vengeance ; elle était si heureuse avant l’incident de la mort de Giletti ! et maintenant, quel était son sort ! elle vivait dans l’attente d’un événement affreux dont elle se serait bien gardée de dire un mot à Fabrice, elle qui autrefois, lors de son arrangement avec Ferrante, croyait tant réjouir Fabrice en lui apprenant qu’un jour il serait vengé. 
 Prediction: The Duchessa had less than never forgotten her vengeance; she was happy before the incident of the mort of Giletti ! and now, which was her sort! she had in theattente of a affrother event which she would be gardée of dire a mot to Fabrizio, she who, when she had an arrangement with Ferrante, croyait tant réjouir Fabrizio en lui apprenant qu'un jour il serait vengeance. 
 Answer: Less than ever had the Duchessa forgotten her revenge; she had been so happy before the incident of Giletti's death --and now, what a fate was hers! She was living in expectation of a dire 